In [1]:
import pandas
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import pandas as pd
import os
from sklearn.model_selection import train_test_split


class Lin_Net(nn.Module):

	def __init__(self):
		# initiate layers
		super(Lin_Net, self).__init__()
		self.lin1 = nn.Linear(8, 8)
		self.lin2 = nn.Linear(8, 4)

	def forward(self, x):
		# define forward pass
		x = F.relu(self.lin1(x))
		x = self.lin2(x)
		return x

	def num_flat_features(self, x):
		self.size = x.size()[1:]
		self.num = 1
		# calculate the number of features
		for i in self.size:
			self.num *= i
		return self.num

In [2]:
def make_data(list_of_datasets): 
	print("generating datasets") 
	datasets = []
	target_vec = []
	data = []
	train_x, train_y, test_x, test_y = [], [], [], []

	# load datasets
	for f in list_of_datasets:
		print("... loading", f)
		datasets.append(pd.read_csv("../" + f))
	dataset = pd.concat(datasets, axis=0, ignore_index=True)
	
	# split data into train and test set
	print("... splitting data")
	train_data, test_data = train_test_split(dataset, test_size=0.3)
	
	# build train set
	print("... seperating target and feature vecs") 	
	for index, row in train_data.iterrows():
		if row["affect"] == 0:
			train_y.append([1, 0, 0, 0])
		elif row["affect"] == 1:
			train_y.append([0, 1, 0, 0])
		elif row["affect"] == 2:
			train_y.append([0, 0, 1, 0])
		elif row["affect"] == 3:
			train_y.append([0, 0, 0, 1]) 
		train_x.append([row["word_count"], row["upper_word_count"], row["ent_word_count"], row["h_count"], row["s_count"], row["a_count"], row["f_count"], row["cons_punct_count"]])


	# build test set	
	for index, row in test_data.iterrows():
		if row["affect"] == 0:
			test_y.append([1, 0, 0, 0])
		elif row["affect"] == 1:
			test_y.append([0, 1, 0, 0])
		elif row["affect"] == 2:
			test_y.append([0, 0, 1, 0])
		elif row["affect"] == 3:
			test_y.append([0, 0, 0, 1]) 
		test_x.append([row["word_count"], row["upper_word_count"], row["ent_word_count"], row["h_count"], row["s_count"], row["a_count"], row["f_count"], row["cons_punct_count"]])

	return Variable(torch.Tensor(train_x)), Variable(torch.Tensor(test_x)), Variable(torch.Tensor(train_y)), Variable(torch.Tensor(test_y))

In [39]:
def train(train_x, train_y):
    for i in range(100):
        print(i)
        inp = torch.Tensor(train_x)
        inp.requires_grad_()
        target = Variable(torch.Tensor(train_y))
        output = lin_net(inp)
        
        criterion = nn.MSELoss()
        loss = criterion(output, target)
        lin_net.zero_grad()
        loss.backward()
        optimizer = optim.SGD(lin_net.parameters(), lr=0.05)
        optimizer.step()
        
        if i == 99:
            print("train loss", loss) 
            print("input", inp)
            print("-->", inp.grad)
        
        
def test(test_x, test_y):
    inp = torch.Tensor(test_x)
    target = Variable(torch.Tensor(test_y))
    output = lin_net(inp)

    criterion = nn.MSELoss()
    loss = criterion(output, target)
    print("test loss", loss)

In [8]:
# load datasats
train_x, test_x, train_y, test_y = make_data(["emotion_classification_1_clean.csv", "emotion_classification_2_clean.csv", "emotion_classification_3_clean.csv", "emotion_classification_4_clean.csv", "emotion_classification_5_clean.csv", "emotion_classification_6_clean.csv", "emotion_classification_7_clean.csv", "emotion_classification_8_clean.csv"])
lin_net = Lin_Net()
lin_net.load_state_dict(torch.load("net_lin_emotion_all.pt"))

generating datasets
... loading emotion_classification_1_clean.csv
... loading emotion_classification_2_clean.csv
... loading emotion_classification_3_clean.csv
... loading emotion_classification_4_clean.csv
... loading emotion_classification_5_clean.csv
... loading emotion_classification_6_clean.csv
... loading emotion_classification_7_clean.csv
... loading emotion_classification_8_clean.csv
... splitting data
... seperating target and feature vecs


<All keys matched successfully>

In [41]:

train(train_x, train_y)
#test(test_x, test_y)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
train loss tensor(0.1756, grad_fn=<MseLossBackward>)
input 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

